# ticket-closure

This notebook contains sample code to build a model which can predictt how long it will take to resolve tickets in an IT support system. It uses the [Incident Management dataset from the UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log).

In [1]:
!pip uninstall -y scipy numpy pandas joblib scikit-learn
!pip install scipy==1.4.1 numpy==1.18.1 pandas==0.24.1 joblib==0.14.1 scikit-learn==0.22.1 lightgbm==2.3.1

Uninstalling scipy-1.1.0:
  Successfully uninstalled scipy-1.1.0
Uninstalling numpy-1.14.3:
  Successfully uninstalled numpy-1.14.3
Uninstalling pandas-0.24.2:
  Successfully uninstalled pandas-0.24.2
Skipping joblib as it is not installed.
Uninstalling scikit-learn-0.20.3:
  Successfully uninstalled scikit-learn-0.20.3
You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/dc/29/162476fd44203116e7980cfbd9352eef9db37c49445d1fec35509022f6aa/scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 20.2MB 2.5MB/s eta 0:00:01
    100% |████████████████████████████████| 10.1MB 6.8MB/s eta 0:00:01
    100% |████████████████████████████████| 296kB 39.5MB/s ta 0:00:01
    100% |████████████████████████████████| 7.1MB 11.4MB/s ta 0:00:01
    100% |████████████████████████████████| 1.2MB 28.5MB/s ta 0:00:01
You are using

In [2]:
import lightgbm as lgb
import numpy as np
import os
import pandas as pd
import pickle
import sklearn
from datetime import datetime

from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

from typing import Dict

To download the training data, run the provided shell script.

```shell
./scripts/get-training-data.sh
```

In [3]:
# load training data
# https://archive.ics.uci.edu/ml/datasets/Incident+management+process+enriched+event+log
TRAIN_FILE = "incident_event_log.csv"

_date_parser = lambda x: pd.NaT if x == '?' else datetime.strptime(x, "%d/%m/%Y %H:%M")
train_df = pd.read_csv(
    TRAIN_FILE,
    parse_dates=[
        "opened_at",
        "resolved_at",
        "closed_at",
        "sys_created_at",
        "sys_updated_at"
    ],
    infer_datetime_format=False,
    converters={
        "opened_at": _date_parser,
        "resolved_at": _date_parser,
        "closed_at": _date_parser,
        "sys_created_at": _date_parser,
        "sys_updated_at": _date_parser
    },
    na_values = ['?']
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (19,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
train_df.head()

# drop columns
#    * stuff you can only know after you've closed the ticket
#    * 'number', which is an incident identifier that is super high-cardinality
#    * high-cardinality columns that are basically obfuscated employee IDs (like 'created_by')
#    * 'caller_id': not creating stateful features like "number of previous tickets from this caller"
drop_cols = [
    "active",
    "assigned_to",
    "caller_id",
    "caused_by",
    "closed_code",
    "incident_state",
    "knowledge",
    "made_sla",
    "number",
    "opened_by",
    "resolved_at",
    "resolved_by",
    "rfc",
    "sys_created_by",
    "sys_updated_by",
    "vendor"
]
train_df = train_df[[col for col in train_df.columns if col not in drop_cols]]
train_df.head()

,reassignment_count,reopen_count,sys_mod_count,opened_at,sys_created_at,sys_updated_at,contact_type,location,category,subcategory,u_symptom,cmdb_ci,impact,urgency,priority,assignment_group,u_priority_confirmation,notify,problem_id,closed_at
0,0,0,0,2016-02-29 01:16:00,2016-02-29 01:23:00,2016-02-29 01:23:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,False,Do Not Notify,NaN,2016-03-05 12:00:00
1,0,0,2,2016-02-29 01:16:00,2016-02-29 01:23:00,2016-02-29 08:53:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,False,Do Not Notify,NaN,2016-03-05 12:00:00
2,0,0,3,2016-02-29 01:16:00,2016-02-29 01:23:00,2016-02-29 11:29:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,False,Do Not Notify,NaN,2016-03-05 12:00:00
3,0,0,4,2016-02-29 01:16:00,2016-02-29 01:23:00,2016-03-05 12:00:00,Phone,Location 143,Category 55,Subcategory 170,Symptom 72,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 56,False,Do Not Notify,NaN,2016-03-05 12:00:00
4,0,0,0,2016-02-29 04:40:00,2016-02-29 04:57:00,2016-02-29 04:57:00,Phone,Location 165,Category 40,Subcategory 215,Symptom 471,NaN,2 - Medium,2 - Medium,3 - Moderate,Group 70,False,Do Not Notify,NaN,2016-03-06 10:00:00


In [5]:
# create target, 'time_to_close'
TARGET_COL = 'time_to_close'
train_df[TARGET_COL] = (train_df['closed_at'] - train_df['opened_at']) / np.timedelta64(1, 's')

In [6]:
from ticket_closure_lib.transformers import OrdinalConverter
from ticket_closure_lib.transformers import DateColTransformer
from ticket_closure_lib.transformers import FeatureRemover

feature_map = {
    "impact": {
        "3 - Low": 1,
        "2 - Medium": 2,
        "1 - High": 3
    },
    "priority": {
        "4 - Low": 1,
        "3 - Moderate": 2,
        "2 - High": 3,
        "1 - Critical": 4
    },
    "urgency": {
        "3 - Low": 1,
        "2 - Medium": 2,
        "1 - High": 3
    }
}
ordinal_transformer = Pipeline(
    steps=[
        ('convert_some_to_int', OrdinalConverter(feature_map=feature_map)),
        ('fill_na', SimpleImputer(strategy="constant", fill_value="placeholder")),
        ('encode', OrdinalEncoder())
    ]
)

numeric_features = [
    col for col in dict(train_df.dtypes).keys()
    if (
        train_df.dtypes[col] == np.dtype('int64') or
        train_df.dtypes[col] == np.dtype('float64')
    )
]
categorical_features = [
    col for col in dict(train_df.dtypes).keys()
    if train_df.dtypes[col] == np.dtype('O')
]

#  https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
# create a feature engineering Python
lgb_estimator = lgb.LGBMRegressor(
    boosting_type='gbdt',
    max_depth=10,
    learning_rate=0.01,
    n_estimators=1000,
    num_leaves = 30,
    objective='regression',
    n_jobs=4,
    silent=False
)

cols_to_drop = [
    "sys_created_at",
    "sys_updated_at",
    "opened_at",
    "closed_at"
]

ordinal_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value="placeholder")),
        ("encode", OrdinalEncoder())
    ]
)

categorical_cols = list(train_df.select_dtypes("O").columns)
pipeline = Pipeline(
    steps=[
        ('date_cols', DateColTransformer()),
        ('remove_intermediate', FeatureRemover(cols_to_drop=cols_to_drop)),
        ('convert_some_to_int', OrdinalConverter(feature_map=feature_map)),
        ('ordinal_col_transformer', ColumnTransformer(
            transformers=[
                ("ordinal", ordinal_transformer, categorical_cols)
            ]
        )),
#         ('fill_na', SimpleImputer(strategy="constant", fill_value="placeholder")),
#         ('encode', OrdinalEncoder()),
        ('regressor', lgb_estimator)
    ]
)

In [7]:
# train-test split
X = train_df[[col for col in train_df.columns if col is not TARGET_COL]]
y = train_df[TARGET_COL]

In [8]:
mod = pipeline.fit(X=X, y=y)

In [9]:
preds = mod.predict(X)

In [10]:
print(f"median ticket duration: {round(train_df[TARGET_COL].median() / (60 * 60 * 24.0), 2)} days")
print(f"MAE: {round(sklearn.metrics.mean_absolute_error(preds, y) / (60.0 * 60.0 * 24.0), 2)} days")
print(f"MSE: {sklearn.metrics.mean_squared_error(preds, y)}")

median ticket duration: 9.37 days
MAE: 9.71 days
MSE: 2903021411202.4043


## Save the model

Now that the model is trained, save it to local storage so it can be used in an application.

In [11]:
LOCAL_MODEL_FILE_NAME = "model.pkl"
with open(LOCAL_MODEL_FILE_NAME, "wb") as f:
    pickle.dump(mod, f)

## Upload to Cloud Storage

At this point in the notebook, we've trained a model but that model only exists on the same machine as this notebook. Let's push it to [Amazon S3](https://aws.amazon.com/s3/) so that we can pull it and re-use it later.

`S3_TRAINING_ARTIFACT_BUCKET` below should be replaced with the name of the bucket you created in CloudFormation. Visit the AWS CloudFormation section of the console to find that name.

In [14]:
import boto3
S3_TRAINING_ARTIFACT_BUCKET = "ticket-closure-model-artifacts-358790040914-us-east-1"
S3 = boto3.resource('s3')
S3.meta.client.upload_file(
    LOCAL_MODEL_FILE_NAME,
    S3_TRAINING_ARTIFACT_BUCKET,
    LOCAL_MODEL_FILE_NAME
)